In [1]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

In [2]:
!pip install keras-image-helper

In [3]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

2022-12-05 12:02:30.260799: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 12:02:30.671578: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-05 12:02:30.671686: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-05 12:02:32.523429: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [4]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [5]:
stub

In [6]:
from keras_image_helper import create_preprocessor

In [7]:
preprocessor = create_preprocessor('xception', target_size=(299,299))

In [8]:
url = "http://bit.ly/mlbookcamp-pants"
X = preprocessor.from_url(url)

In [9]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape = data.shape)

In [10]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'
pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [11]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [12]:
pb_response

outputs {
  key: "dense_7"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: -1.8682903051376343
    float_val: -4.761245250701904
    float_val: -2.316983461380005
    float_val: -1.0625708103179932
    float_val: 9.887161254882812
    float_val: -2.8124334812164307
    float_val: -3.6662826538085938
    float_val: 3.200361728668213
    float_val: -2.6023378372192383
    float_val: -4.835046291351318
  }
}
model_spec {
  name: "clothing-model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

In [13]:
preds = pb_response.outputs['dense_7'].float_val

In [14]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [15]:
dict(zip(classes, preds))

{'dress': -1.8682903051376343,
 'hat': -4.761245250701904,
 'longsleeve': -2.316983461380005,
 'outwear': -1.0625708103179932,
 'pants': 9.887161254882812,
 'shirt': -2.8124334812164307,
 'shoes': -3.6662826538085938,
 'shorts': 3.200361728668213,
 'skirt': -2.6023378372192383,
 't-shirt': -4.835046291351318}